In [49]:
import os
import folium

from freeman.aiddd.data_manager import read_data

#### VWorld Setting

In [50]:
BASE_KEY_FILE_PATH = os.path.join(
    os.path.expanduser('~'), 'projects', 'data'
)
VWORLD_KEY_FILE_PATH = os.path.join(BASE_KEY_FILE_PATH, 'vworld_key.txt')
with open(VWORLD_KEY_FILE_PATH, 'r') as vkf:
    vk_lines = vkf.readlines()
VWORLD_KEY = vk_lines[0].replace('\n', '')

VWORLD_TILE_TYPE='png'
VWORLD_ATTR = 'Vworld'

In [2]:
df_data = read_data('3rd pp pole-position-on-cons-1st')

In [4]:
MAX_POLE_COUNTS = 10
dict_positions = {}

In [6]:
for _, row in df_data.iterrows():
    key = row.cons_no
    cons_no_path = []
    for idx in range(1, MAX_POLE_COUNTS+1):
        column_name_x, column_name_y = f'pole{idx}_x', f'pole{idx}_y'
        if row[column_name_x] == 0.0:
            break
        cons_no_path.append([row[column_name_x], row[column_name_y]])
    dict_positions[key] = cons_no_path

In [7]:
dict_positions['477420183473']

[[128.39612467324, 37.092508235338],
 [128.396295207885, 37.0929025747383],
 [128.396333799759, 37.0933041440521],
 [128.396304930086, 37.0938037611575],
 [128.396835889318, 37.0941065932202],
 [128.396767948465, 37.0945963812899],
 [128.394117445502, 37.0900161668407],
 [128.394374045407, 37.0900875447172],
 [128.394464774386, 37.090044004806]]

In [56]:
LABEL_POSITION = 0.0001
def show_pole_and_line(cons_no, VWORLD_LAYER='Base'):
    path = dict_positions[cons_no]
    x_values = [point[1] for point in path]
    y_values = [point[0] for point in path]
    min_x, max_x = min(x_values), max(x_values)
    min_y, max_y = min(y_values), max(y_values)
    
    VWORLD_TILES_URL = \
    f'http://api.vworld.kr/req/wmts/1.0.0/' \
    f'{VWORLD_KEY}/{VWORLD_LAYER}/{{z}}/{{y}}/{{x}}.{VWORLD_TILE_TYPE}'
    
    base_map = folium.Map(
        location = [(min_x + max_x)/2, (min_y + max_y)/2],
        tiles=VWORLD_TILES_URL, attr=VWORLD_ATTR, zoom_start=16
    )
    
    poly_path = [[y, x] for x, y in path]
    folium.PolyLine(locations=poly_path, color='green').add_to(base_map)
    for idx, point in enumerate(path):
        folium.CircleMarker(location=[point[1],point[0]], radius=3, color='blue').add_to(base_map)
        folium.Marker(
            location=[point[1]+LABEL_POSITION,point[0]+LABEL_POSITION],
            icon=folium.DivIcon(
                html=f"<div style='font-size: 14px; color: red;'>{idx+1}</div>"
            )
        ).add_to(base_map)
    return base_map

In [52]:
show_pole_and_line('477420204328') # 길 코너를 따르지 않음(1-2-3 구간), 6-7-8구간 이상

In [57]:
show_pole_and_line('477420204329', 'Hybrid')   # 길과 전혀 상관없는 곳에서 공사함

In [58]:
show_pole_and_line('477420213346', 'midnight')  # 전산화번호가 공사 순서가 아닌가? (GISID가 공사순서인가?)

In [59]:
show_pole_and_line('477420213362', 'white')     # 1-2-3 구간이 이해가 안감